# Setup Paths

In [ ]:
from PIL import Image, ImageDraw
import os
from pathlib import Path
import albumentations
import numpy as np
import xml.etree.ElementTree as ET
import math


In [ ]:
# path = "Enter the path to the model directory: "
# dir_path = "C:\\Users\\--\\Documents\\uot16"
dir_path = "MODEL DIR PATH"
folder_holding_dataset = os.path.join(dir_path, "preprocessing", "dataset")
counter = 0
count = 0

# Resize

In [ ]:
def resize_image(img_arr, bboxes, h, w):
    """
    :param img_arr: original image as a numpy array
    :param bboxes: bboxes as numpy array where each row is 'x_min', 'y_min', 'x_max', 'y_max', "class_id"
    :param h: resized height dimension of image
    :param w: resized weight dimension of image
    :return: dictionary containing {image:transformed, bboxes:['x_min', 'y_min', 'x_max', 'y_max', "class_id"]}
    """
    # create resize transform pipeline
    transform = albumentations.Compose(
        [albumentations.Resize(height=h, width=w, always_apply=True)],
        bbox_params=albumentations.BboxParams(format='pascal_voc'))

    transformed = transform(image=img_arr, bboxes=bboxes)
    print(transformed["bboxes"])
    
    return transformed

In [ ]:
# get class names from dataset directory
classes = [c.name for c in os.scandir(folder_holding_dataset) if c.is_dir()]

for c in classes:
  print(c) 

  # get path of each class directory
  class_folder = os.path.join(folder_holding_dataset, c)

  # get file names from class directory
  files = [f.name for f in os.scandir(class_folder) if f.is_file()]

  for f in files:
    # get path of each file 
    file_path = os.path.join(class_folder, f)

    # # files are named as integers (ex. 1-800)
    # # gets integer 
    # count=int(Path(file_path).stem)
    # print(count)

    # # gets image path
    # img_path =   os.path.join(class_folder, str(count)+".jpg")
    # # gets annotation path
    # xml_path =   os.path.join(class_folder, str(count)+".xml")

    # gets image path
    img_path =   os.path.join(class_folder, Path(f).stem+".jpg")
    # gets annotation path
    xml_path =   os.path.join(class_folder, Path(f).stem+".xml")

    im = Image.open(img_path)
    # display(im)

    # create numpy array
    im = np.asarray(im)

    # gets xml info so we can modify
    tree = ET.parse(xml_path)
    root = tree.getroot()
    sample_annotations = []
    # get image width and height from xml
    for neighbor in root.iter('size'):
      width = int(neighbor.find('width').text)
      height = int(neighbor.find('height').text)

    # get bounding box data from xml
    for neighbor in root.iter('bndbox'):
      # if original annotations fall outside actual image, we correct
      # ~1 pixel area around border of image where model will detect poorly
      xmin = max(int(float(neighbor.find('xmin').text)), 1)
      ymin = max(int(float(neighbor.find('ymin').text)), 1)
      xmax = min(int(float(neighbor.find('xmax').text)), int(width)-1)
      ymax = min(int(float(neighbor.find('ymax').text)), int(height)-1)
      
      # this error value will help tell us if annotation values are equal/incorrect
      allowed_error = 1.0
      # print(count)
      
      # checks for the allowed error; checks if annotations need correction
      if abs(xmin - xmax) <= allowed_error:
        xmin = math.floor(xmin) 
        xmax = math.ceil(xmax)
        if xmin == xmax:
          xmin -= 1
          xmax += 1 

      # checks for the allowed error; checks if annotations need correction    
      if abs(ymin - ymax) <= allowed_error:
        ymin = math.floor(ymin) 
        ymax = math.ceil(ymax)
        if ymin == ymax:
          ymin -= 1
          ymax += 1
        
      # collect bounding box and class annotations  
      sample_annotations.append([round(xmin), round(ymin), round(xmax), round(ymax), c])
    sample_annotations = np.asarray(sample_annotations, dtype = object)  
    print(img_path)
    print(sample_annotations)
    
    # transform the image using the sample annotations; pass desired resize arguments
    transformed_im = resize_image(im, sample_annotations, 320, 320)
    trans_im = Image.fromarray(transformed_im["image"])
    draw = ImageDraw.Draw(trans_im)

    # to plot bounding boxes:
    # for bbox in transformed_im["bboxes"]:
      # print(type(bbox[0:4]))
      # draw.rectangle(list(bbox[0:4]), outline= "red")
      # print("here")

    # print(transformed_im)
    # display(trans_im)
    
    print(transformed_im["bboxes"])

    # write new annotations to the xml files
    the_file = xml_path
    tree = ET.parse(the_file)
    root = tree.getroot()
    bbox_idx = 0
    for box in root.find("object").findall("bndbox"):
      child_idx = 0
      for child in box:
        # print(child.text)
        child.text = str(int(transformed_im["bboxes"][bbox_idx][child_idx]))
        child_idx+=1
      bbox_idx+=1
    tree.write(the_file)
    trans_im.save(img_path)
